In [1]:
import requests
import json
import pandas as pd
import gzip

In [2]:
base_url = "https://api.themoviedb.org/3"

In [3]:
# Faz requisição para API, tendo como padrão a API_KEY e a linguagem
def make_request(endpoint, additional_params = None):
    default_params = {'api_key': '3ca27b067484f1867da24107d9ed9222', 'language': 'pt-BR'}
    if additional_params:
      default_params.update(additional_params)
    req = requests.get(base_url + endpoint, params=default_params)
    return req

## Funções para moantar datasets (Tasks)

Essa função  responsável por transformar os dados retornados pela API em um dataframe contendo apenas as features selecionadas e criadas para as necessidades do projeto

In [4]:
# Trata o dataset de filmes ou séries, padronizando nomes, adaptando e criando novas features
def make_media_dataframe(movies_dict):
  final_df = pd.DataFrame.from_dict(movies_dict)

  final_df.set_index('id', inplace=True)
  
  # Padroniza os nomes das features para filmes e séries
  final_df.rename(columns={
    'name': 'title',
    'original_name': 'original_title',
    'first_air_date': 'release_date'
  }, inplace=True, errors='ignore')
  
  final_df.drop(['adult', 'backdrop_path', 'original_language', 'video'], inplace=True, axis=1, errors='ignore')
  
  final_df['poster_path'] = 'https://image.tmdb.org/t/p/w220_and_h330_face' + final_df['poster_path']
  
  final_df['release_date'] = pd.to_datetime(final_df['release_date'])
  final_df['release_year'] = final_df['release_date'].dt.year
  
  return final_df

Essa função é responsável por fazer a requisição na rota de detalhes de um filme ou série a fim de obter dados mais específicos sobre ele.
O campo a ser retornado (`detail_fields`) pode ser uma string especificando o nome do campo ou um dicionário no qual é especifica uma função que, executada no valor origianal daquele campo (`fieldName`, retornado pela API) gera o valor final desejado no campo de chave `finalName`.

In [5]:
def complete(media_type, media, detail_fields):
  endpoint = f'/{media_type}/{media["id"]}'
  res = make_request(endpoint)
  data = {}
  values = {}
  if res.ok:
    try:
      data = json.loads(res.text)
    except:
      data = {}
    for field in detail_fields:
      try:
        if type(field) == str:
          values[field] = data[field]
        elif type(field) == dict:
          values[field['finalName']] = field['func'](data[field['fieldName']])
      except (KeyError):
        fieldName = field['finalName'] if type(field) == dict else field
        values[fieldName] = None
      
    media.update(values)

Função responsável por recuperar dados da rota trending para filmes ou para séries

In [6]:
def get_trending(trending_type, time_interval, detail_fields, pages):
    endpoint = f'/trending/{trending_type}/{time_interval}'
    
    return make_media_list(endpoint, pages, trending_type, detail_fields, {})

Fução responsável por formar uma lista com filmes ou séries a partir dos dados retornados por uma rota. Ela também itera pelas páginas especificadas no parâmetro `pages`

In [7]:
def make_media_list(endpoint, pages, media_type, detail_fields, params):
    response = make_request(endpoint, params)

    if not response.ok or not response.text:
      raise Exception('Error in the request')

    data = json.loads(response.text)

    if data['total_results'] == 0:
      return []

    final_list = data['results']
    pages = min(pages, data['total_pages'])
    
    final_list.extend(iter_pages(endpoint, 2, pages, params))

    if detail_fields:
      for media in final_list:
        complete(media_type, media, detail_fields)

    return final_list

Função para iterar pelas páginas de uma resposta

In [8]:
def iter_pages(endpoint, intial_page, end_page, params = None):
  results = []
  
  for i in range (intial_page, end_page + 1):
    page_param =  {'page': i}
    
    if params:
      page_param.update(params)

    page_param.update(params)
    
    response = make_request(endpoint, page_param)
    data = json.loads(response.text)
    results.extend(data['results'])

  return results

# Dataset de gêneros

In [9]:
endpoint_genres_tv = '/genre/tv/list'
endpoint_genres_movie = '/genre/movie/list'
genres_tv_res = make_request(endpoint_genres_tv)
genres_movie_res = make_request(endpoint_genres_movie)
genres_movie = json.loads(genres_movie_res.text)['genres']
genres_tv = json.loads(genres_tv_res.text)['genres']

Montagem de uma lista de gêneros a partir da qual os IDS de gênero presentes no retorno da API sejam mapeados para palavras. A partir dessa lista, iremos alterar alguns gêneros para termos o resultado esperado na aplicação final

In [10]:
unique_genres = list(set([tuple(d.items()) for d in genres_movie + genres_tv]))
unique_genres = [dict(genre) for genre in unique_genres]

In [11]:
genres_map = {}
for genre in unique_genres:
  genres_map[genre['id']] = genre['name']

In [12]:
pd.Series(index=map(lambda i: i['id'], unique_genres), data=map(lambda i: i['name'], unique_genres))

12                 Aventura
10402                Música
10751               Família
10752                Guerra
16                 Animação
10765      Sci-Fi & Fantasy
10763                  News
10767                  Talk
10768        War & Politics
10764               Reality
36                 História
28                     Ação
9648               Mistério
10762                  Kids
14                 Fantasia
10749               Romance
37                 Faroeste
27                   Terror
10770             Cinema TV
18                    Drama
99             Documentário
878       Ficção científica
80                    Crime
53                 Thriller
10766                  Soap
35                  Comédia
10759    Action & Adventure
dtype: object

In [13]:
genres_map[10766] = 'Novela'
genres_map[10765] = ['Ficção científica', 'Fantasia']
genres_map[10762] = 'Infantil'
genres_map[10768] = ['Guerra', 'Política']
genres_map[10759] = ['Ação', 'Aventura']

In [14]:
excluded = 10765, 10768, 10759
added = ['Política']
new_genres = []
for key, genre in genres_map.items():
  if key not in excluded:
    new_genres.append({'id': abs(hash(genre)), 'name': genre})
for genre in added:
  new_genres.append({'id': abs(hash(genre)), 'name': genre})

In [15]:
new_genres_df = pd.DataFrame.from_dict(new_genres).set_index('id')

In [16]:
new_genres_df

,name
id,
875186253494820635,Aventura
4605734590823903241,Música
379447581407342637,Família
3676917530199795176,Guerra
7478426263258974621,Animação
3776198685149603334,News
4834731283544586559,Talk
1461893468257324033,Reality
7491433743285069447,História


## Análise das features

Primeira questão: quais features dos filmes iremos manter
  - id
  - poster
  - id
  - title
  - original_title
  - overview
  - poster_path: deixar a URL inteira
  - vote_average
  - vote_count
  - popularity
  - release_date
  - media_type
  - genre: transformar ids em nomes

# Dataset de filmes

Função que transforma IDS de gêneros nas respostas da API nos nomes adotados dutante o processo de tratamento de dados

In [17]:
def map_genres(movie, genres):
  # Mapear os gêneros
  movie['genres'] = []
  for genre_id in movie['genre_ids']:
    if type(genres[genre_id]) == list:
      for g in genres[genre_id]:
        movie['genres'].append(g)
    else:
      movie['genres'].append(genres[genre_id])
  movie.pop('genre_ids')
  return movie

In [18]:
def serialize(movies, genres_map):
  return list(map(lambda movie: map_genres(movie, genres_map), movies))

In [19]:
movie_details = [
    'imdb_id', 
    'status', 
    'revenue', 
    'budget',
    {'fieldName': 'belongs_to_collection', 'finalName': 'collection', 'func': lambda x: x['name'] if x else None},
    {'fieldName': 'production_companies', 'finalName': 'productions', 'func': lambda prod: [x['name'] for x in prod]}
]
trending_movies = get_trending('movie', 'week', movie_details, 3)
trending_movies_formatted = serialize(trending_movies, genres_map)

In [20]:
movies = make_media_dataframe(trending_movies_formatted)

In [21]:
movies.head()

,title,original_title,overview,poster_path,media_type,popularity,release_date,vote_average,vote_count,imdb_id,status,revenue,budget,collection,productions,genres,release_year
id,,,,,,,,,,,,,,,,,
436270,Adão Negro,Black Adam,Quase 5.000 anos depois de ter sido concedido ...,https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,5089.869,2022-10-19,7.310,2653,tt6443346,Released,384571691,200000000,None,"[New Line Cinema, Flynn Picture Company, Seven...","[Ação, Fantasia, Ficção científica]",2022
736526,O Troll da Montanha,Troll,Após ficar aprisionada por mil anos nas profun...,https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,4232.311,2022-12-01,6.816,483,tt11116912,Released,0,0,None,[Motion Blur],"[Fantasia, Ação, Aventura]",2022
774752,Guardiões da Galáxia: Especial de Festas,The Guardians of the Galaxy Holiday Special,Com a missão de tornar o Natal inesquecível pa...,https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,955.212,2022-11-25,7.479,694,tt13623136,Released,0,0,None,"[Marvel Studios, Troll Court Entertainment, Ke...","[Comédia, Ficção científica, Aventura]",2022
715931,Emancipação: Uma História de Liberdade,Emancipation,"Peter, um escravo em fuga é forçado a despista...",https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,82.506,2022-12-02,7.598,66,tt12530246,Released,0,0,None,"[Escape Artists, Fuqua Films, Apple, Westbrook...","[História, Drama, Mistério, Thriller, Ação]",2022
555604,Pinóquio por Guillermo Del Toro,Guillermo del Toro's Pinocchio,Uma versão mais sombria do clássico conto de f...,https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,222.841,2022-11-09,8.400,177,tt1488589,Released,0,35000000,None,"[The Jim Henson Company, ShadowMachine, Double...","[Animação, Fantasia, Drama]",2022


In [22]:
movies.head()

,title,original_title,overview,poster_path,media_type,popularity,release_date,vote_average,vote_count,imdb_id,status,revenue,budget,collection,productions,genres,release_year
id,,,,,,,,,,,,,,,,,
436270,Adão Negro,Black Adam,Quase 5.000 anos depois de ter sido concedido ...,https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,5089.869,2022-10-19,7.310,2653,tt6443346,Released,384571691,200000000,None,"[New Line Cinema, Flynn Picture Company, Seven...","[Ação, Fantasia, Ficção científica]",2022
736526,O Troll da Montanha,Troll,Após ficar aprisionada por mil anos nas profun...,https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,4232.311,2022-12-01,6.816,483,tt11116912,Released,0,0,None,[Motion Blur],"[Fantasia, Ação, Aventura]",2022
774752,Guardiões da Galáxia: Especial de Festas,The Guardians of the Galaxy Holiday Special,Com a missão de tornar o Natal inesquecível pa...,https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,955.212,2022-11-25,7.479,694,tt13623136,Released,0,0,None,"[Marvel Studios, Troll Court Entertainment, Ke...","[Comédia, Ficção científica, Aventura]",2022
715931,Emancipação: Uma História de Liberdade,Emancipation,"Peter, um escravo em fuga é forçado a despista...",https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,82.506,2022-12-02,7.598,66,tt12530246,Released,0,0,None,"[Escape Artists, Fuqua Films, Apple, Westbrook...","[História, Drama, Mistério, Thriller, Ação]",2022
555604,Pinóquio por Guillermo Del Toro,Guillermo del Toro's Pinocchio,Uma versão mais sombria do clássico conto de f...,https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,222.841,2022-11-09,8.400,177,tt1488589,Released,0,35000000,None,"[The Jim Henson Company, ShadowMachine, Double...","[Animação, Fantasia, Drama]",2022


# Dataset de séries

In [23]:
detail_fields = [
    'in_production',
    'episode_run_time',
    'number_of_seasons',
    'number_of_seasons',
    'number_of_episodes',
    {'fieldName' : 'networks', 'finalName': 'networks', 'func': (lambda networks: [n['name'] for n in networks])}
]
trending_tv = get_trending('tv', 'week', detail_fields, 2)
trending_tv_formatted = serialize(trending_tv, genres_map)

In [24]:
tv_shows = make_media_dataframe(trending_tv_formatted)

In [25]:
tv_shows.head()

,title,original_title,overview,poster_path,media_type,popularity,release_date,vote_average,vote_count,origin_country,in_production,episode_run_time,number_of_seasons,number_of_episodes,networks,genres,release_year
id,,,,,,,,,,,,,,,,,
119051,Wandinha,Wednesday,"Inteligente, sarcástica e apática, Wandinha Ad...",https://image.tmdb.org/t/p/w220_and_h330_face/...,tv,7390.878,2022-11-23,8.779,2533,[US],True,[],1,8,[Netflix],"[Ficção científica, Fantasia, Mistério, Comédia]",2022
111837,Willow,Willow,Anos após os eventos do filme “Willow” origina...,https://image.tmdb.org/t/p/w220_and_h330_face/...,tv,1699.615,2022-11-30,5.796,49,[US],True,[50],1,8,[Disney+],"[Ficção científica, Fantasia, Ação, Aventura, ...",2022
90669,1899,1899,Em um navio a vapor migrante rumo ao oeste par...,https://image.tmdb.org/t/p/w220_and_h330_face/...,tv,835.311,2022-11-17,7.929,493,[DE],True,[50],1,8,[Netflix],"[Mistério, Drama]",2022
95403,Periféricos,The Peripheral,Flynne Fisher É uma garota que tenta juntar os...,https://image.tmdb.org/t/p/w220_and_h330_face/...,tv,732.854,2022-10-20,8.100,397,[US],True,[60],1,8,[Amazon],"[Ficção científica, Fantasia, Drama]",2022
83867,Star Wars: Andor,Star Wars: Andor,As aventuras do espião rebelde Cassian Andor d...,https://image.tmdb.org/t/p/w220_and_h330_face/...,tv,273.298,2022-09-21,8.114,398,[US],True,[40],1,12,[Disney+],"[Ficção científica, Fantasia, Ação, Aventura, ...",2022


# Dataset de people

In [26]:
def map_known_for(person):
  if person['known_for']:
    person['known_for_name'] = [movie.get('title') or movie.get('name') for movie in person['known_for']]
    person['known_for_id'] = [movie['id'] for movie in person['known_for']]

In [27]:
def serialize_people(people):
  return list(map(lambda person: map_known_for(person), people))

In [28]:
def make_trending_people_df(trending_people):
  people_df = pd.DataFrame(trending_people)
  people_df.drop('known_for', axis=1, inplace=True)
  people_df.set_index('id', inplace=True)
  people_df['profile_path'] = 'https://image.tmdb.org/t/p/w220_and_h330_face' + people_df['profile_path']
  
  return people_df

In [29]:
trending_people = get_trending('person', 'week', ['imdb_id'], 2)
trending_people_formatted = serialize_people(trending_people)

In [30]:
trending_people_df = make_trending_people_df(trending_people)

In [31]:
trending_people_df

,adult,name,original_name,media_type,popularity,gender,known_for_department,profile_path,imdb_id,known_for_name,known_for_id
id,,,,,,,,,,,
3714087,False,Yua Mikami,Yua Mikami,person,4.753,1,Acting,https://image.tmdb.org/t/p/w220_and_h330_face/...,None,"[SKE48 Spring Concert 2012, SKE48 Request Hour...","[1028092, 1031684, 1033456]"
974169,False,Jenna Ortega,Jenna Ortega,person,409.959,1,Acting,https://image.tmdb.org/t/p/w220_and_h330_face/...,nm4911194,"[Wandinha, X: A Marca da Morte, Pânico]","[119051, 760104, 646385]"
2888,False,Will Smith,Will Smith,person,35.199,2,Acting,https://image.tmdb.org/t/p/w220_and_h330_face/...,nm0000226,"[Esquadrão Suicida, Eu Sou a Lenda, Eu, Robô]","[297761, 6479, 2048]"
3453397,False,雷火剑,雷火剑,person,3.392,0,Directing,NaN,None,NaN,NaN
500,False,Tom Cruise,Tom Cruise,person,27.480,2,Acting,https://image.tmdb.org/t/p/w220_and_h330_face/...,nm0000129,"[No Limite do Amanhã, Oblivion, Missão: Imposs...","[137113, 75612, 56292]"
287,False,Brad Pitt,Brad Pitt,person,47.876,2,Acting,https://image.tmdb.org/t/p/w220_and_h330_face/...,nm0000093,"[Bastardos Inglórios, Seven: Os Sete Crimes Ca...","[16869, 807, 72190]"
62,False,Bruce Willis,Bruce Willis,person,41.116,2,Acting,https://image.tmdb.org/t/p/w220_and_h330_face/...,nm0000246,"[O Sexto Sentido, Duro de Matar, O Quinto Elem...","[745, 562, 18]"
18897,False,Jackie Chan,Jackie Chan,person,44.324,2,Acting,https://image.tmdb.org/t/p/w220_and_h330_face/...,nm0000329,"[A Hora do Rush, A Hora do Rush 2, A Hora do R...","[2109, 5175, 5174]"
6193,False,Leonardo DiCaprio,Leonardo DiCaprio,person,52.956,2,Acting,https://image.tmdb.org/t/p/w220_and_h330_face/...,nm0000138,"[A Origem, Titanic, Ilha do Medo]","[27205, 597, 11324]"


## Visualização detalhada de filmes e séries

Função que retorna um dataset formado a partir de uma string de busca específico. A ideia é que **ela pode ser usada com os termos mais encontrado pelo Crawler**

In [32]:
def query(media_type, query, pages, detail_fields, genres_map):
  endpoint = f'/search/{media_type}'
  params = {'query': query}
  
  final_list = make_media_list(endpoint, pages, media_type, detail_fields, params)
  
  formatted_list = serialize(final_list, genres_map)
  final_df = make_media_dataframe(final_list)

  
  return final_df
  

In [33]:
results = query('movie', 'Matrix', 2, movie_details, genres_map)

# Top rated

In [34]:
def get_route(route, media_type, detail_fields, pages, genres_map, params = None):
    endpoint = f'/{media_type}/{route}'
    
    media_list = make_media_list(endpoint, pages, media_type, detail_fields, params)
    formatted_list = serialize(media_list, genres_map)
    return make_media_dataframe(formatted_list)

In [35]:
# popular_movies = get_route('popular', 'movie', movie_details, 2, genres_map)

In [36]:
# popular_movies.head()

NameError: name 'popular_movies' is not defined

In [ ]:
popular_tv = get_route('popular', 'tv', detail_fields, 2, genres_map)

In [ ]:
popular_tv.head()

In [ ]:
top_rated_movies = get_route('top_rated', 'movie', movie_details, 5, genres_map)

In [ ]:
top_rated_movies.head()

In [ ]:
upcoming = get_route('upcoming', 'movie', movie_details, 1, genres_map, {'region': 'BR'})

In [ ]:
upcoming.head()

# Lidando com o `daily export file`

Esse arquivo é exportado diariamente para o link ```http://files.tmdb.org/p/exports/movie_ids_[mm]_[dd]_[yyyy].json.gz``` com uma série de `ids` válidos

In [ ]:
def get_daily_movies(day, month, n, details):
  day = str(day).zfill(2)
  month = str(month).zfill(2) 
  url = f'https://files.tmdb.org/p/exports/movie_ids_{month}_{day}_2022.json.gz'

  response = requests.get(url, stream=True)
  daily_movies = []

  try:
    with gzip.GzipFile(fileobj=response.raw,  mode='rb') as daily_file:
      for i in range(n):
        movie = json.loads(daily_file.readline())
        complete('movie', movie, details)
        daily_movies.append(movie)

      daily_movies_formatted = serialize(daily_movies, genres_map)
      daily = make_media_dataframe(daily_movies_formatted)
      return daily
  except FileNotFoundError as e:
      print('Arquivo não encontrado')


In [ ]:
extraDetails = movie_details + [
      {'fieldName': 'genres', 'finalName': 'genre_ids', 'func': lambda genres: [g['id'] for g in genres]},
      'poster_path',
      'release_date',
  ]
daily_20 = get_daily_movies(day=22, month=11, n=20, details=extraDetails)

In [ ]:
daily_20.shape

In [ ]:
daily_20.head()

## Filtrar filmes pela data de lançamento

In [42]:
def get_movies_in_date_range(initial_year, final_year, movie_details, pages, sorting = 'asc'):
  endpoint = '/discover/movie'
  initial_date = f'{initial_year}-01-01'
  final_date = f'{final_year}-12-31'
  params = {'primary_release_date.gte': initial_date, 'primary_release_date.lte': final_date, 'sort_by': f'primary_release_date.{sorting}'}
  results = make_media_list(endpoint, pages, 'movie', movie_details, params)
  serialized = serialize(results, genres_map)
  return make_media_dataframe(serialized)

In [43]:
# cada "pages" tem 20 filmes, entao se pegamos pages=10, pegamos 10pages*20filmes = 200 filmes
df = get_movies_in_date_range(2020, 2021, movie_details, 200, 'desc')

In [44]:
df

,original_title,overview,popularity,poster_path,release_date,title,vote_average,vote_count,imdb_id,status,revenue,budget,collection,productions,genres,release_year
id,,,,,,,,,,,,,,,,
1056995,Вовка и зима в Тридевятом царстве,,0.600,https://image.tmdb.org/t/p/w220_and_h330_face/...,2021-12-31,Вовка и зима в Тридевятом царстве,4.0,1,None,Released,0,0,None,[],[Animação],2021
1054751,Rescuing a Fantasy Classic,,0.600,NaN,2021-12-31,Rescuing a Fantasy Classic,7.0,1,None,Released,0,0,None,[Signal Hill Entertainment],[Documentário],2021
1049732,Diomar,,0.600,NaN,2021-12-31,Diomar,0.0,0,tt13598760,Released,0,0,None,[],[Documentário],2021
1039528,Bien parecidas,,0.600,https://image.tmdb.org/t/p/w220_and_h330_face/...,2021-12-31,Bien parecidas,0.0,0,None,Released,0,0,None,"[Villanos Cine, Ácidas Films, Fire Pit Studios]",[Drama],2021
1037342,Hay algo que no he dicho,,0.600,https://image.tmdb.org/t/p/w220_and_h330_face/...,2021-12-31,Hay algo que no he dicho,0.0,0,None,Released,0,0,None,[],[],2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898029,Engomado,,0.600,https://image.tmdb.org/t/p/w220_and_h330_face/...,2021-11-20,Engomado,0.0,0,None,Released,0,0,None,[],[],2021
898028,Érase una vez en Quizca,,0.600,https://image.tmdb.org/t/p/w220_and_h330_face/...,2021-11-20,Érase una vez en Quizca,0.0,0,tt13176588,Released,0,0,None,[Cabeza Negra Cine],[Documentário],2021
898027,Las máquinas tristes,,0.600,https://image.tmdb.org/t/p/w220_and_h330_face/...,2021-11-20,Las máquinas tristes,0.0,0,None,Released,0,0,None,[],[Documentário],2021


In [46]:
df.to_csv("data_output/TMDB_2021_4000.csv", index=False)